In [1]:
import wikipedia
wikipedia.set_lang('ru')

In [2]:
url = wikipedia.page('Список городов России').url

Парсим страницу Википедии со списком всех городов Росии

In [3]:
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import pandas as pd
import re
import pickle

In [4]:
gen_page = requests.get(url)
soup = BeautifulSoup(gen_page.text,'lxml')
print(soup.prettify()[0:99])

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="ru">
 <head>
  <meta charset="utf-8"/>
  


In [5]:
tabel = soup.find('table', {'class':'standard sortable'})
data = {}
t = '1'
for i in tabel.find_all('a'):
    
    title = i.get('title')
    link = i.get('href')
    
    if t == '':
        title = re.sub(r'\([^\)]+\)', '', title).rstrip()
        data[title] = link
    t = i.text

In [6]:
# создание выборки для тестирования работы алгоритма
#simple_data = {}

#for n, l in tqdm(data.items()):
#    simple_data[n] = l
#    if len(simple_data) > 15:
#        break
#data = simple_data
#print(len(data))

In [7]:
#функция для получения данных о населении и принадлежности города к субъекту РФ
def get_features(url):
    url_city = 'https://ru.wikipedia.org' + url
    soup1 = BeautifulSoup(requests.get(url_city).text,'lxml')
    years = []
    pops = []
    population = soup1.find_all('table', {'class':'standard'})
   
    if len(population) > 1:
        for i in range(len(population)):
            try:
                p = population[i].find('th').text
            except:
                continue
            if p == 'Численность населения':
                population = population[i]
                break
    else:
        population = population[0]
        
    
    for i in population.find_all('th'):
        year = i.text[:4]
        if year.isdigit() == True:
            years.append(year)
        
    for i in population.find_all('td'):
        pop = ''.join([x for x in i.text if x.isdigit()])
        if pop != '':
            pops.append(int(pop))
    
    feats_city = dict(zip(years, pops))
    other_data = soup1.find('table', {'class':'infobox vcard'})
    
    if type(other_data) == type(None):
        return feats_city

    feat = []
    val = []
    
    for i in other_data.find_all('th'):
        feat.append(i.text)
    
    for i in other_data.find_all('td'):
        d = ''.join([line.lstrip('\n\xa0') for line in i.text])
        val.append(d)

    sub_f, sub_v = [], []
    
    for i in range(len(feat)):
        if feat[i] == 'Страна':
            sub_f = feat[i:i+4]
            break
            
    for i in range(len(val)):
        if val[i] == 'Россия':
            sub_v = val[i:i+4]
            break
            
    for f, v in zip(sub_f, sub_v):
        if f == 'Субъект Федерации':
            feats_city[f] = v
       
    return feats_city

In [8]:
df = {}
err = []
for n, l in tqdm(data.items()):
    try:
        df[n] = get_features(l)
    except Exception:
        err.append(n)
        print(n, Exception.__class__)
        

Санкт-Петербург <class 'type'>
Тольятти <class 'type'>
Электроугли <class 'type'>



Данные по населению Москвы и Санкт-Петербурга представлены на отдельных страницах

In [9]:
splash = {'Москва':'/wiki/Население_Москвы',
          'Санкт-Петербург':'/wiki/Население_Санкт-Петербурга'
         }

splash_df = {}

for n, l in tqdm(splash.items()):
    try:
        splash_df[n] = get_features(l)
    except Exception:
        err.append(n)
        print(n, Exception.__class__)
        
splash_df['Москва']['Субъект Федерации'] = 'Москва'
splash_df['Санкт-Петербург']['Субъект Федерации'] = 'Санкт-Петербург'
        
df.update(splash_df)


In [23]:
frame = pd.DataFrame.from_dict(df, orient='index')

In [24]:
frame.head()

,1959,1970,1979,1989,1996,1998,2000,2001,2002,2003,...,1695,1650,1625,1827,1846,1951,1632,1955,1765,1919
Абаза,11646.0,15202.0,15818.0,17630.0,18500.0,18600.0,18700.0,18700.0,18052.0,18000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Абакан,56416.0,90136.0,128311.0,154092.0,162000.0,167000.0,169200.0,167900.0,165197.0,165200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Абдулино,29976.0,26010.0,23054.0,22639.0,23600.0,23900.0,NaN,24000.0,21537.0,21500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Абинск,14750.0,21222.0,25251.0,29182.0,34100.0,34500.0,34900.0,35000.0,33734.0,33700.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Агрыз,20270.0,19267.0,20137.0,19732.0,19400.0,19800.0,NaN,NaN,18620.0,18600.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Удаляем города со спорной принадлежностью к РФ

In [25]:
for_drop = frame[pd.isna(frame['Субъект Федерации']) == True]['Субъект Федерации']
print(for_drop)
frame.drop(for_drop.index, axis=0, inplace=True)

Алупка             NaN
Алушта             NaN
Бахчисарай         NaN
Белогорск          NaN
Джанкой            NaN
Евпатория          NaN
Керчь              NaN
Красноперекопск    NaN
Курильск           NaN
Нюрба              NaN
Саки               NaN
Севастополь        NaN
Симферополь        NaN
Старый Крым        NaN
Судак              NaN
Феодосия           NaN
Ялта               NaN
Армянск            NaN
Щёлкино            NaN
Name: Субъект Федерации, dtype: object


In [26]:
frame.columns

Index(['1959', '1970', '1979', '1989', '1996', '1998', '2000', '2001', '2002',
       '2003',
       ...
       '1695', '1650', '1625', '1827', '1846', '1951', '1632', '1955', '1765',
       '1919'],
      dtype='object', length=337)

Переводим название городов и субъектов РФ на английский язык

In [27]:
#from googletrans import Translator
#eng_names = []

#for c in tqdm(frame.index):
    
#    translator = Translator(service_urls=[
#      'translate.google.com',
#      'translate.google.ru',
#    ])
#    eng = translator.translate(c, src='ru', dest='en').text
#    eng_names.append(eng)
#    print(c, eng)

Google translater выдаёт ошибки при работе, а также не переводит некоторые названия. Для решения данной проблемы учтём, что перевод большинства населённых пунктов осуществляется на основании транслитерации.

In [28]:
rus_chars = [c.lower() for c in
            ['А','Б','В','Г','Д','Е','Ё','Ж',
             'З','И','Й','К','Л','М','Н','О',
             'П','Р','С','Т','У','Ф','Х','Ц',
             'Ч','Ш','Щ','Ъ','Ы','Ь','Э','Ю',
             'Я', ' ','-']]
eng_chars = [c.lower() for c in
            ['A','B','V','G','D','E','E','ZH',
             'Z','I','I','K','L','M','N','O',
             'P','R','S','T','U','F','KH','TS',
             'CH','SH','SHCH','\"','Y','\'',
             'E','IU','IA', ' ','-']]



In [29]:
def translit(word):
    if word == 'Москва':
        return 'Мoscow'
    if word == 'Санкт-Петербург':
        return 'Saint-Petersburg'
    
    translit = []
    for c in word.lower():
        for r, e in zip(rus_chars, eng_chars):
            if c==r:
                translit.append(e)
                break
            #else:
                #translit.append(c)
                #break
    return ''.join(translit).capitalize()

In [30]:
eng_names = []

for c in tqdm(frame.index):
    eng_names.append(translit(c))

frame['City'] = eng_names

In [31]:
eng_reg = []

for c in tqdm(frame['Субъект Федерации']):
    eng_reg.append(translit(c))

frame['Subject of Federation'] = eng_reg
frame.drop(['Субъект Федерации'], axis=1, inplace=True)

Сортируем признаки

In [34]:
frame = frame[sorted(frame.columns, key=lambda x: x)]
frame.head()

,1350,1400,1550,1557,1562,1571,1574,1590,1594,1595,...,2013,2014,2015,2016,2017,2018,2019,2020,City,Subject of Federation
Абаза,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,16491.0,16238.0,16009.0,15802.0,15592,15335.0,NaN,14990.0,Abaza,Khakasiia
Абакан,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,169760.0,173205.0,176212.0,179163.0,181709,184168.0,186201.0,186797.0,Abakan,Khakasiia
Абдулино,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,19687.0,19468.0,19353.0,19320.0,19222,19032.0,18768.0,18581.0,Abdulino,Orenburgskaia oblast'
Абинск,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,35921.0,36607.0,36986.0,37415.0,37749,38176.0,38547.0,39058.0,Abinsk,Krasnodarskii krai
Агрыз,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,19671.0,19778.0,19738.0,19739.0,19704,19774.0,19794.0,19721.0,Agryz,Tatarstan


Проверяем на наличие данных с пропусками за последние 2018 и 2020 годы

In [35]:
frame[pd.isna(frame['2018']) == True][pd.isna(frame['2020']) == True].head(100)

<ipython-input-35-f8d2901e9f70>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  frame[pd.isna(frame['2018']) == True][pd.isna(frame['2020']) == True].head(100)


,1350,1400,1550,1557,1562,1571,1574,1590,1594,1595,...,2013,2014,2015,2016,2017,2018,2019,2020,City,Subject of Federation
Руза,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,13733.0,13554.0,13419.0,13393.0,13403,NaN,NaN,NaN,Ruza,Moskovskaia oblast'


Cохраняем данные

In [36]:
def csv_writer(data, name):
    data.to_csv(name, index=False)
csv_writer(frame, 'population_of_the_Russian_cities.csv')

In [37]:
with open('dictionary with links.pickle', 'wb') as f:
    pickle.dump(data, f)